In [ ]:
import json
import subprocess
import time
from tqdm import tqdm

def load_yorumlar(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return [item.get("comment", "") for item in data["reviews"]]

def ollama_etiketle(yorum, model="mistral"):
    prompt = f"""
Sen bir duygu analiz modelisin. Aşağıdaki Türkçe kullanıcı yorumunu okuduktan sonra, yalnızca yorumun duygusunu temsil eden bir sayı döndür.

Etiket kuralları:
- 1 → Pozitif
- 0 → Nötr
- 2 → Negatif

Örnekler:
• "Ürün harika, çok memnun kaldım." → 1
• "Beklediğim gibi geldi, fena değil." → 0
• "Bozuk geldi, iade ettim." → 2

Şimdi şu yorumu değerlendir:
"{yorum}"

Sadece bir sayı döndür. Açıklama yazma.
Cevap:
""".strip()

    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt.encode("utf-8"),
            capture_output=True,
            timeout=60
        )
        output = result.stdout.decode("utf-8").strip()
        for token in output.split():
            if token.strip() in ["0", "1", "2"]:
                return int(token.strip())
    except Exception as e:
        print(f"⚠️ Hata: {e}")
    return 0


def toplu_etiketle(yorumlar):
    etiketli = []
    for yorum in tqdm(yorumlar):
        etiket = ollama_etiketle(yorum)
        etiketli.append({"comment": yorum, "label": etiket})
        time.sleep(1)  # Ollama'yı yormamak için beklet
    return etiketli

def kaydet_json(data, cikis_yolu="etiketli_yorumlar_mistral.json"):
    with open(cikis_yolu, "w", encoding="utf-8") as f:
        json.dump({"reviews": data}, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    yorumlar = load_yorumlar("response.json")
    etiketli_yorumlar = toplu_etiketle(yorumlar)
    kaydet_json(etiketli_yorumlar)
    print("\n✅ Etiketleme tamamlandı. 'etiketli_yorumlar_mistral.json' dosyasına kaydedildi.")


100%|██████████| 101/101 [01:54<00:00,  1.14s/it]


✅ Etiketleme tamamlandı. 'etiketli_yorumlar_mistral.json' dosyasına kaydedildi.
